In [ ]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
test_df = pd.read_csv("Test_Dataset.csv")

In [ ]:
test_df.drop('Unnamed: 19', axis=1, inplace=True)
test_df.drop('Unnamed: 20', axis=1, inplace=True)

In [ ]:
test_df['location_code'].fillna(test_df['location_code'].mode()[0], inplace=True)

In [ ]:
def preprocess_data(df):
  # filling the NaN values of international plan with the mode
  df['intertiol_plan'].fillna(df['intertiol_plan'].mode()[0], inplace=True)

  df.loc[((df['voice_mail_plan'] == 'no') | (df['voice_mail_plan'].isna())) & (df['number_vm_messages'] != 0), 'number_vm_messages'] = 0.0
  df.loc[(df['voice_mail_plan'].isna()) & (df['number_vm_messages'] == 0), 'voice_mail_plan'] = 'no'
  df.loc[(df['voice_mail_plan'] == 'yes') & (df['number_vm_messages'].isna()), 'number_vm_messages'] = 0

  cols_with_neg = ['total_day_min', 'total_day_charge', 'total_day_calls', 'total_eve_min', 'total_eve_calls', 'total_night_minutes', 'total_intl_minutes', 'total_intl_calls']
  for col in cols_with_neg:
    df[col] = np.abs(df[col])


  charge_per_min_day = np.mean(df['total_day_charge']/df['total_day_min'])
  df.loc[df['total_day_min'].isna(), 'total_day_min'] = df['total_day_charge']/charge_per_min_day
  df.loc[df['total_day_charge'].isna(), 'total_day_charge'] = df['total_day_min']*charge_per_min_day

  avg_call_dur_day = np.mean(df['total_day_min']/df['total_day_calls'])
  df.loc[df['total_day_calls'].isna(), 'total_day_calls'] = np.round(df['total_day_min']/avg_call_dur_day, 0)


  charge_per_min_eve = np.mean(df['total_eve_charge']/df['total_eve_min'])
  df.loc[df['total_eve_min'].isna(), 'total_eve_min'] = df['total_eve_charge']/charge_per_min_eve
  df.loc[df['total_eve_charge'].isna(), 'total_eve_charge'] = df['total_eve_min']*charge_per_min_eve

  avg_call_dur_eve = np.mean(df['total_eve_min']/df['total_eve_calls'])
  df.loc[df['total_eve_calls'].isna(), 'total_eve_calls'] = np.round(df['total_eve_min']/avg_call_dur_eve, 0)


  charge_per_min_night = np.mean(df['total_night_charge']/df['total_night_minutes'])
  df.loc[df['total_night_minutes'].isna(), 'total_night_minutes'] = df['total_night_charge']/charge_per_min_night
  df.loc[df['total_night_charge'].isna(), 'total_night_charge'] = df['total_night_minutes']*charge_per_min_night

  avg_call_dur_night = np.mean(df['total_night_minutes']/df['total_night_calls'])
  df.loc[df['total_night_calls'].isna(), 'total_night_calls'] = np.round(df['total_night_minutes']/avg_call_dur_night, 0)


  charge_per_min_intl = np.mean(df['total_intl_charge']/df['total_intl_minutes'])
  df.loc[df['total_intl_minutes'].isna(), 'total_intl_minutes'] = df['total_intl_charge']/charge_per_min_intl
  df.loc[df['total_intl_charge'].isna(), 'total_intl_charge'] = df['total_intl_minutes']*charge_per_min_intl

  intl_calls = df[(df['total_intl_calls']>0)]
  avg_call_dur_intl = np.mean(intl_calls['total_intl_minutes']/intl_calls['total_intl_calls'])
  df.loc[df['total_intl_calls'].isna(), 'total_intl_calls'] = np.round(df['total_intl_minutes']/avg_call_dur_intl, 0)


  df.loc[df['total_intl_calls']==0, ['total_intl_minutes', 'total_intl_charge']] = 0.0

  df.loc[df['customer_service_calls'].isna(), 'customer_service_calls'] = 0

In [ ]:
preprocess_data(chatterbox)
preprocess_data(test_df)

In [ ]:
test_df.loc[(test_df['total_day_min'].isna()) & (test_df['total_day_charge'].isna()), ['total_day_min', 'total_day_calls', 'total_day_charge']] = 0

In [ ]:
test_df.to_csv("Test_Dataset_preprocessed.csv", index=False)